Taken from https://github.com/thohemp/6DRepNet.

The head pose detector needs pre cropped images, so we will use MTCNN. The reasoning behind this is because it is the same detector that they used to produce a training dataset so hopefully it will produce good results here.

In [1]:
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display

from mtcnn import MTCNN

from SixDRepNet import SixDRepNet

face_detector = MTCNN()
pose_detector = SixDRepNet()

def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg',value)[1])

def resize_with_padding(image,target_size):
    old_size= image.shape[:2]
    print(old_size)
    ratio = float(target_size) / max(old_size)
    new_dim = (int(old_size[1] * ratio),int(old_size[0] * ratio))
    print(new_dim)

    image = cv2.resize(image, new_dim)

    return image


2022-11-15 10:54:20.535507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 10:54:20.861608: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jacob/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-15 10:54:20.861636: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-15 10:54:20.922488: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already be

In [2]:
img = cv2.imread("images/image 02.jpg")


detection = face_detector.detect_faces(img)

print("Detected {} faces".format(str(len(detection))))

for face in detection:
    bbox = face["box"]
    print(bbox)
    # create raw bounding box in blue
    img = cv2.rectangle(img,
    (bbox[0],bbox[1]),
    (bbox[0] + bbox[2],bbox[1] + bbox[3]),
    (255,127,0),
    2)

    # create square centered bounding box in orange

    box_size = max(bbox[2:4])
    top_left = (int(bbox[0] + (bbox[2]/2) - (box_size/2)),int(bbox[1] + (bbox[3]/2) - (box_size/2)))
    print(top_left)
    
    img = cv2.rectangle(img,
    top_left,
    (top_left[0] + box_size,top_left[1] + box_size),
    (0,127,255),
    2)

    cropped = img[top_left[1]:top_left[1]+box_size,top_left[0]:top_left[0]+box_size]
    print(cropped.shape)

image_widget = widgets.Image(format='jpeg', width=512, height=512)
image_widget.value = bgr8_to_jpeg(img)
display(image_widget)

1/1 [==============================] - 0s 102ms/step
Detected 1 faces
[449, 172, 295, 378]
(407, 172)
(378, 378, 3)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [3]:
pitch, yaw, roll = pose_detector.predict(cropped)

print(pitch, yaw, roll)

pose_detector.draw_axis(img, yaw, pitch, roll,int(top_left[0] + box_size/2),int(top_left[1] + box_size/2))

image_widget.value = bgr8_to_jpeg(img)
display(image_widget)

[-13.152084] [0.9569228] [0.58643585]


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [ ]:
video_capture = cv2.VideoCapture()

video_capture.set()